# Creating Templates with [Jinja2](http://jinja.pocoo.org/)

> _Jinja2 is a modern and designer-friendly templating language for Python, modelled after Django’s templates. It is fast, widely used and secure with the optional sandboxed template execution environment._

In the previous notebook, you've seen how common Flask functions work and we made a tempalte render on the screen. In this notebook we'll take a deeper look into templates.

In [ ]:
%load templates/hello.html

# Templates

We didn't import the Jinja2 module in our code directly since Flask takes care of this for us. The details of using the jinja2 module are hidden by Flask. We can focus on learning how the template works and the template language to create pages that our server will render.

Looking at the template's source code, it seems like normal HTML interspersed with odd braces that contain `{{ variables or expressions }}` and `{% statements %}`. Have a closer look at it. These are just some of the delimiters available in the template. Various statement __tags__ are available for use. We'll learn about those in a bit. The template language easy because it's similar to Python so there's only a few things to learn.


## [Comments](http://jinja.pocoo.org/docs/dev/templates/#comments)

If you need to put comments in your template, just enclose it with `{#  #}`.


## [Variables](http://jinja.pocoo.org/docs/dev/templates/#variables)

`flask.render_template()` accepts a template name and keyword arguments for the context. Those keyword arguments become your variables within the template. Variables can be of different types and can be manipulated like how you typically would in Python, with only a few additional features.

### [Filters](http://jinja.pocoo.org/docs/dev/templates/#builtin-filters)

> _Variables can be modified by filters. Filters are separated from the variable by a pipe symbol (|) and may have optional arguments in parentheses. Multiple filters can be chained. The output of one filter is applied to the next._

Example:

```
{{ name|striptags|title }}
```

### Accessing attributes via dot ( `.` )

The following lines do the same thing:

```
{{ foo.bar }}
{{ foo['bar'] }}
```

## [Expressions](http://jinja.pocoo.org/docs/dev/templates/#expressions)

> _Jinja allows basic expressions everywhere. These work very similarly to regular Python; even if you’re not working with Python you should feel comfortable with it._

Expressions in templates can accept most of the things you can do in Python:

- Literals
- Math
- Comparison
- Other operators
- even `If expressions`

Examples:

```
{{ 1 + 1 }}
{{ '[%s]' % page.title if page.title }}
```

## [Statements](http://jinja.pocoo.org/docs/dev/templates/#line-statements)

The following two examples are equivalent:

```
<ul>
{% for item in seq %}
    <li>{{ item }}</li>
{% endfor %}
</ul>
```

and:

```
<ul>
# for item in seq
    <li>{{ item }}</li>
# endfor
</ul>
```

Line statements can span multiple lines if there are open parentheses, braces or brackets:

```
<ul>
# for href, caption in [('index.html', 'Index'),
                        ('about.html', 'About')]:
    <li><a href="{{ href }}">{{ caption }}</a></li>
# endfor
</ul>
```

Line-based comments are also available since v2.2:

```
# for item in seq:
    <li>{{ item }}</li>  ## this comment is ignored
# endfor
```

# [Template Inheritance](http://jinja.pocoo.org/docs/dev/templates/#template-inheritance)

> _The most powerful part of Jinja is template inheritance. Template inheritance allows you to build a base “skeleton” template that contains all the common elements of your site and defines blocks that child templates can override._

## Base Template

In this example, the `{% block %}` tags define four blocks that child templates can fill in. All the block tag does is tell the template engine that a child template may override those placeholders in the template.

__`base.html`__:

```
<!DOCTYPE html>
<html lang="en">
<head>
    {% block head %}
    <link rel="stylesheet" href="style.css" />
    <title>{% block title %}{% endblock %} - My Webpage</title>
    {% endblock %}
</head>
<body>
    <div id="content">{% block content %}{% endblock %}</div>
    <div id="footer">

        {% block footer %}
        &copy; Copyright 2008 by <a href="http://domain.invalid/">you</a>.
        {% endblock %}

    </div>
</body>
</html>
```

## Child Template

The {% extends %} tag is the key here. It tells the template engine that this template “extends” another template. When the template system evaluates this template, it first locates the parent. The extends tag should be the first tag in the template. Everything before it is printed out normally and may cause confusion.

__`index.html`__:

```
{% extends "base.html" %}

{% block title %}Index{% endblock %}

{% block head %}
    {{ super() }}
    <style type="text/css">
        .important { color: #336699; }
    </style>
{% endblock %}

{% block content %}
    <h1>Index</h1>
    <p class="important">
      Welcome to my awesome homepage.
    </p>
{% endblock %}
```

> _Example taken from the documentation._

There are a couple of things here. We have two templates, `base.html` and `index.html`. In both templates, we have __`{% block %}`__ tags with their own names. Our child template, `index.html`, has an __`{% extends %}`__ tag. This tells Flask to look for the template with the specified name first, in this case, `base.html`, and that this template is an extension of that one. This is where the named __`{% blocks %}`__ come in. The content of the child template's block tags go into the base template's block tags with the same name. The child template inherits from the base template and fills in the blocks.

## [Include](http://jinja.pocoo.org/docs/dev/templates/#include)

Another way to make a template reusable is to write the component in a separate file. This will __not__ use __`{% extend %}`__. Instead, the __`{% include %}`__ tag can be used in any other template. The specified template will be inserted into that template and have access to the variables within the current template's scope.

# Control Structures

Control structures are also available in the template langugage. These tags allow you to create assignments, apply filters, use loops and conditionals, even macros, among other things. It's still works mostly like Python but with some adjustments.

## [Assignment](http://jinja.pocoo.org/docs/dev/templates/#assignments) - `{% set %}`


> _Inside code blocks, you can also assign values to variables. Assignments at top level (outside of blocks, macros or loops) are exported from the template like top level macros and can be imported by other templates._

> _Assignments use the set tag and can have multiple targets:_

```
{% set navigation = [('index.html', 'Index'), ('about.html', 'About')] %}
{% set key, value = call_something() %}
```

## [Filters](http://jinja.pocoo.org/docs/dev/templates/#id11) - `{% filter %}`

> _Filter sections allow you to apply regular Jinja2 filters on a block of template data. Just wrap the code in the special filter section:_

```
{% filter upper %}
    This text becomes uppercase
{% endfilter %}
```

## [Conditionals](http://jinja.pocoo.org/docs/dev/templates/#if) - `{% if %}`

Template conditionals are also similar to Python conditionals:

```
{% if kenny %}

    {% if kenny.sick %}
        Kenny is sick.
    {% elif kenny.dead %}
        You killed Kenny!  You bastard!!!
    {% else %}
        Kenny looks okay --- so far
    {% endif %}

{% endif %}
```

## [Loops](http://jinja.pocoo.org/docs/dev/templates/#for) - `{% for %}`

Template loops are pretty much similar to Python loops, with only a few differences. Within a for-loop block, a special loop variable is accessible:

![loop](./Images/loop.png)

> _Unlike in Python, it’s not possible to break or continue in a loop. You can, however, filter the sequence during iteration, which allows you to skip items. The following example skips all the users which are hidden:_

```
{% for user in users if not user.hidden %}
    <li>{{ user.username|e }}</li>
{% endfor %}
```

> _If no iteration took place because the sequence was empty or the filtering removed all the items from the sequence, you can render a default block by using else:_

```
<ul>
{% for user in users %}
    <li>{{ user.username|e }}</li>
{% else %}
    <li><em>no users found</em></li>
{% endfor %}
</ul>
```

> _Note that, in Python, else blocks are executed whenever the corresponding loop did not break. Since Jinja loops cannot break anyway, a slightly different behavior of the else keyword was chosen._

## [Macro](http://jinja.pocoo.org/docs/dev/templates/#macros) and [Call](http://jinja.pocoo.org/docs/dev/templates/#call)

There are also ways to define functions in the templates with macro and call for programming reusable behavior.

#### Exercise:

Create a Flask app:

- Theme is an upcoming real or imaginary product
- Create a Product class with the following attributes:
    - Name
    - Features (at least 3)
    - Release date
- Name is a string, Features is a list of strings, Release date is a datetime.date instance
- For now, values are hardcoded


- Only one page
- Page should prominently display the name of the product
- Page should list the features of the product
- When the date is equal to or after the release date, a "Buy now" button should appear (It can link to anywhere)


- Page should include a "Subscribe to our mailing list" form with the field:
    - Email address
- Page should include a "Contact us" form with the fields:
    - Name
    - Email address
    - Subject
    - Content
- Submitted form data should be validated accordingly.
- Invalid forms should not be accepted and return the respective errors.
- Valid forms should print the submitted info to the console.
- Page should flash a message confirming the submitted form